# Assumptions

## 1. There are no measurement errors.

If we had to account for measurement errors then we would have to run multiple surface codes cycles to infer where the error has occured. There are efficient methods that work for sparse errors but if the error probabilities are high then surface codes cannot correct the errors.

# Imports

In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
import numpy as np
import matplotlib.pyplot as plt

# Parameters

In [2]:
surface_code_sq = 5

In [3]:
if surface_code_sq%2 ==0:
    raise Exception('surface_code_sq should be odd')

In [4]:
# n,k,d
n = (surface_code_sq**2)//2 + 1

# Logical Qubit

In [5]:
data_qubits = QuantumRegister(n, 'd')
ancilla_qubits_1 = QuantumRegister((n-1)//2, 'a1')
ancilla_qubits_2 = QuantumRegister((n-1)//2, 'a2')
syndrome_cregs_1 = ClassicalRegister((n-1)//2, 's1')
syndrome_cregs_2 = ClassicalRegister((n-1)//2, 's2')

In [6]:
data_qubits_index = 0
ancilla_qubits_1_index = 0
ancilla_qubits_2_index = 0

logical_qubit_array = []

for i in range(surface_code_sq):
    if i%2 == 0:
        for j in range(surface_code_sq//2):
            logical_qubit_array.append(data_qubits[data_qubits_index])
            data_qubits_index += 1
            logical_qubit_array.append(ancilla_qubits_1[ancilla_qubits_1_index])
            ancilla_qubits_1_index += 1
        logical_qubit_array.append(data_qubits[data_qubits_index])
        data_qubits_index += 1
    else:
        for j in range(surface_code_sq//2):
            logical_qubit_array.append(ancilla_qubits_2[ancilla_qubits_2_index])
            ancilla_qubits_2_index += 1
            logical_qubit_array.append(data_qubits[data_qubits_index])
            data_qubits_index += 1
        logical_qubit_array.append(ancilla_qubits_2[ancilla_qubits_2_index])
        ancilla_qubits_2_index += 1

In [7]:
logical_qubit_array = np.array(logical_qubit_array, dtype=QuantumRegister)

In [8]:
logical_qubit_2d = logical_qubit_array.reshape(surface_code_sq, surface_code_sq)

In [9]:
ancilla_qubits_1_neighbours = { }
ancilla_qubits_2_neighbours = { }

for row_index in range(0, surface_code_sq, 2):
    for col_index in range(1, surface_code_sq, 2):
        ancilla = logical_qubit_2d[row_index, col_index]
        neighbours = []
        for i, j in ((-1, 0), (0, -1), (0, 1), (1, 0)):
            data_qubit_row_index = row_index + i
            data_qubit_col_index = col_index + j
            if data_qubit_row_index > -1 and data_qubit_col_index > -1 and data_qubit_row_index < surface_code_sq and data_qubit_col_index < surface_code_sq:
                data_qubit = logical_qubit_2d[data_qubit_row_index, data_qubit_col_index]
                neighbours.append(data_qubit)
        ancilla_qubits_1_neighbours[ancilla] = neighbours

for row_index in range(1, surface_code_sq, 2):
    for col_index in range(0, surface_code_sq, 2):
        ancilla = logical_qubit_2d[row_index, col_index]
        neighbours = []
        for i, j in ((-1, 0), (0, -1), (0, 1), (1, 0)):
            data_qubit_row_index = row_index + i
            data_qubit_col_index = col_index + j
            if data_qubit_row_index > -1 and data_qubit_col_index > -1 and data_qubit_row_index < surface_code_sq and data_qubit_col_index < surface_code_sq:
                data_qubit = logical_qubit_2d[data_qubit_row_index, data_qubit_col_index]
                neighbours.append(data_qubit)
        ancilla_qubits_2_neighbours[ancilla] = neighbours
                

We no longer need the 2d array `logical_qubit_2d` because we created the neighbours dictionaries

In [10]:
ancilla_qubits_2_neighbours[ancilla_qubits_2[0]]

[Qubit(QuantumRegister(13, 'd'), 0),
 Qubit(QuantumRegister(13, 'd'), 3),
 Qubit(QuantumRegister(13, 'd'), 5)]

In [11]:
del logical_qubit_2d

# Test circ

In [12]:
def surface_code_encode(qc: QuantumCircuit,
                        x_ancilla_qubits_neighbours: dict[QuantumRegister, list[QuantumRegister]],
                        z_ancilla_qubits_neighbours: dict[QuantumRegister, list[QuantumRegister]]):
    z_ancilla_qubits = list(z_ancilla_qubits_neighbours.keys())
    for x_ancilla, neighbour_data_qubits in x_ancilla_qubits_neighbours.items():
        for neighbour_data_qubit in neighbour_data_qubits:
            qc.cx(x_ancilla, neighbour_data_qubit)

    qc.h(z_ancilla_qubits)
    for z_ancilla, neighbour_data_qubits in z_ancilla_qubits_neighbours.items():
        for neighbour_data_qubit in neighbour_data_qubits:
            qc.cx(z_ancilla, neighbour_data_qubit)

In [13]:
def surface_code_measure_syndrome(qc: QuantumCircuit, x_ancilla_qubits: QuantumRegister, z_ancilla_qubits: QuantumRegister,
                                  x_syndrome_cregs: ClassicalRegister, z_syndrome_cregs: ClassicalRegister):
    qc.barrier()
    qc.measure(x_ancilla_qubits, x_syndrome_cregs)
    qc.measure(z_ancilla_qubits, z_syndrome_cregs)


In [14]:
qc = QuantumCircuit(data_qubits, ancilla_qubits_1, ancilla_qubits_2, syndrome_cregs_1, syndrome_cregs_2)
surface_code_encode(qc=qc,
                    x_ancilla_qubits_neighbours=ancilla_qubits_1_neighbours,
                    z_ancilla_qubits_neighbours=ancilla_qubits_2_neighbours)
surface_code_measure_syndrome(qc=qc,
                              x_ancilla_qubits=ancilla_qubits_1,
                              z_ancilla_qubits=ancilla_qubits_2,
                              x_syndrome_cregs=syndrome_cregs_1,
                              z_syndrome_cregs=syndrome_cregs_2)